# Dự đoán giá nhà sử dụng Hồi quy, Ridge, Neural Network và Stacking

In [ ]:

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


## 1. Tiền xử lý dữ liệu

In [ ]:

# Đọc dữ liệu từ file
df = pd.read_csv('/ML_BTL/dataset4.csv')

# Chia dữ liệu thành features (biến độc lập) và target (biến phụ thuộc)
X = df.drop(columns=['PRICE'])  # Các đặc trưng
y = df['PRICE']  # Biến mục tiêu là giá nhà

# Kiểm tra và xử lý giá trị thiếu (nếu có)
if df.isnull().sum().any():
    X.fillna(X.mean(), inplace=True)

# Chia dữ liệu thành tập huấn luyện (80%) và tập kiểm tra (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## 2. Hồi quy tuyến tính (Linear Regression)

In [ ]:

# Huấn luyện mô hình Linear Regression
linear_regressor = LinearRegression()
linear_regressor.fit(X_train_scaled, y_train)

# Dự đoán
y_pred_linear = linear_regressor.predict(X_test_scaled)

# Đánh giá mô hình
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

print(f"Linear Regression - Mean Squared Error (MSE): {mse_linear}")
print(f"Linear Regression - R² Score: {r2_linear}")


## 3. Hồi quy Ridge (Ridge Regression)

In [ ]:

# Huấn luyện mô hình Ridge Regression
ridge_regressor = Ridge(alpha=1.0)
ridge_regressor.fit(X_train_scaled, y_train)

# Dự đoán
y_pred_ridge = ridge_regressor.predict(X_test_scaled)

# Đánh giá mô hình
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print(f"Ridge Regression - Mean Squared Error (MSE): {mse_ridge}")
print(f"Ridge Regression - R² Score: {r2_ridge}")


## 4. Mạng nơ-ron (Neural Network - MLPRegressor)

In [ ]:

# Huấn luyện mô hình MLP Regressor
mlp_regressor = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=2000, learning_rate_init=0.0005, random_state=42)
mlp_regressor.fit(X_train_scaled, y_train)

# Dự đoán
y_pred_mlp = mlp_regressor.predict(X_test_scaled)

# Đánh giá mô hình
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
r2_mlp = r2_score(y_test, y_pred_mlp)

print(f"MLP Regressor - Mean Squared Error (MSE): {mse_mlp}")
print(f"MLP Regressor - R² Score: {r2_mlp}")


## 5. Stacking Regressor

In [ ]:

# Xây dựng mô hình Stacking
base_models = [
    ('linear', LinearRegression()),
    ('ridge', Ridge(alpha=1.0)),
    ('mlp', MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=2000, learning_rate_init=0.0005, random_state=42))
]

final_model = Ridge()

# Stacking Regressor
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=final_model)
stacking_regressor.fit(X_train_scaled, y_train)

# Dự đoán
y_pred_stacking = stacking_regressor.predict(X_test_scaled)

# Đánh giá mô hình
mse_stacking = mean_squared_error(y_test, y_pred_stacking)
r2_stacking = r2_score(y_test, y_pred_stacking)

print(f"Stacking Regressor - Mean Squared Error (MSE): {mse_stacking}")
print(f"Stacking Regressor - R² Score: {r2_stacking}")


## 6. Đánh giá và lưu kết quả

In [ ]:

# Tạo DataFrame để theo dõi quá trình đánh giá mô hình
evaluation_df = pd.DataFrame(columns=['Model', 'MSE', 'R² Score'])

def add_to_evaluation(model_name, mse, r2):
    evaluation_df.loc[len(evaluation_df)] = [model_name, mse, r2]

# Thêm kết quả của từng mô hình vào DataFrame
add_to_evaluation('Linear Regression', mse_linear, r2_linear)
add_to_evaluation('Ridge Regression', mse_ridge, r2_ridge)
add_to_evaluation('MLP Regressor', mse_mlp, r2_mlp)
add_to_evaluation('Stacking Regressor', mse_stacking, r2_stacking)

# Xuất kết quả ra file CSV
evaluation_df.to_csv('model_evaluation.csv', index=False)

# Hiển thị kết quả
print(evaluation_df)


## 7. Vẽ biểu đồ Giá thực tế vs Giá dự đoán

In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Giá thực tế', color='blue', marker='o')
plt.plot(y_pred_stacking, label='Giá dự đoán (Stacking)', color='red', marker='x')
plt.title('Giá thực tế vs Giá dự đoán')
plt.xlabel('Mẫu')
plt.ylabel('Giá nhà')
plt.legend()
plt.show()
